# Imports

In [1]:
#!pip install statsmodels matplotlib

In [2]:
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, DoubleType
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import numpy as np
import numpy as np
from pyspark.sql.functions import col
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.types import DoubleType
import time
import scipy.stats as stats

# Initialize Spark session

In [3]:
spark = SparkSession.builder.master("spark://spark-master:7077").appName("ManualQRDecomposition").getOrCreate()
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/27 16:52:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 1) Funktionen für Lineare Regression mit QR Decomposition

## 1.1) Lösen eines LGS durch Rückwärts Einsetzen

In [4]:
def backward_substitution(R, b):
    """
    Solves the upper triangular system Ax = y for x using backward substitution.
    
    Args:
    R: Upper triangular matrix
    y (array): The right-hand side vector.
    
    Returns:
    array: Solution vector b.
    """
    n = len(b)
    x = np.zeros(n)
    
    for i in range(n-1, -1, -1):
        sum_val = 0
        for j in range(i+1, n):
            sum_val += R[i, j] * x[j]
        
        if abs(R[i, i]) > 1e-10:  # Check for numerical stability
            x[i] = (b[i] - sum_val) / R[i, i]
        else:
            x[i] = 0
            
    return x

## 1.2) QR Zerlegung

### 1.2.1) Gram Schmidt ohne Sparkoptimierung

In [5]:
def manual_qr_decomposition(X_array):
    """
    Manual implementation of QR decomposition using Gram-Schmidt process.
    
    Args:
        X_array: Input matrix as numpy array
        
    Returns:
        Q: Orthogonal matrix
        R: Upper triangular matrix
    """
    n, m = X_array.shape
    Q = np.zeros((n, m))
    R = np.zeros((m, m))
    
    for j in range(m):
        v = X_array[:, j].copy()
        
        for i in range(j):
            R[i, j] = 0
            for k in range(n):
                R[i, j] += Q[k, i] * X_array[k, j]
            
            for k in range(n):
                v[k] = v[k] - R[i, j] * Q[k, i]
        
        norm = 0
        for k in range(n):
            norm += v[k] * v[k]
        norm = np.sqrt(norm)
        
        R[j, j] = norm
        
        if norm > 1e-10:  # Avoid division by very small numbers
            for k in range(n):
                Q[k, j] = v[k] / norm
        else:
            for k in range(n):
                Q[k, j] = 0
    
    return Q, R

### 1.2.1) Gram Schmidt mit Sparkoptimierung

In [6]:
def gram_schmidt(X_df):
    X_rdd = X_df.rdd.cache()
    m = len(X_df.first().features)
    Q = []
    R = np.zeros((m, m))

    for j in range(m):
        v = X_rdd.map(lambda row: row.features[j]).collect()
        
        for i in range(j):
            R[i, j] = np.dot(Q[i], v)
            v -= R[i, j] * Q[i]

        R[j, j] = np.linalg.norm(v)
        Q.append(v / R[j, j])
    
    R_dict = {i: R[i, :] for i in range(m)}

    Q_df = spark.createDataFrame([Row(features=DenseVector(q)) for q in zip(*Q)])

    return Q_df, R_dict

## 1.3) Datensatz simulieren

### 1.3.1) Datensatz simulieren ohne Sparkoptimierung

In [7]:
def create_data_numpy(n, p, beta_true):
    np.random.seed(42)
    X = np.random.rand(n, p)
    X = np.column_stack([np.ones(X.shape[0]), X])
    y = X @ beta_true + np.random.randn(n) * 0.1
    return X, y

### 1.3.2) Datensatz simulieren mit Sparkoptimierung

In [8]:
def create_data_spark(n_samples, n_features, beta_true, noise_std=0.1, partition_size=10000):
    """
    Generates synthetic data optimized for distributed processing.
    
    Args:
        spark: SparkSession
        n: Number of observations
        p: Number of features
        beta_true: True coefficients
        num_partitions: Number of partitions for distributed data
    """
    def generate_partition(partition_size):
        X = np.random.randn(partition_size, n_features)
        X = np.column_stack([np.ones(partition_size), X])
        y = X @ beta_true + np.random.normal(0, noise_std, partition_size)
        return [(Vectors.dense(x), float(y_i)) for x, y_i in zip(X, y)]
    
    num_partitions = max(n_samples // partition_size, spark.sparkContext.defaultParallelism)
    samples_per_partition = n_samples // num_partitions
    
    rdd = (spark.sparkContext
           .parallelize(range(num_partitions), num_partitions)
           .flatMap(lambda _: generate_partition(samples_per_partition)))
    
    return spark.createDataFrame(rdd, ["features", "y"])

## 1.4) Funktion zur Durchführung der linearen Regression 

- simuliert einen Datensatz
- führt die lineare Regression durch
- berechnet alle Metriken, die auch statmodels `summary()` ausgibt
- misst die Zeit

### 1.4.1) Funktion zur Berechnung der Statistik

In [9]:
def compute_statistics(X, y, beta, residuals):
    n, k = X.shape
    SSE = np.sum(residuals ** 2)
    SST = np.sum((y - np.mean(y)) ** 2)
    SSR = SST - SSE
    df_residuals = n - k
    df_model = k - 1
    
    r_squared = 1 - (SSE / SST)
    adj_r_squared = 1 - ((1 - r_squared) * (n - 1) / df_residuals)
    
    MSE = SSE / df_residuals
    MSR = SSR / df_model
    f_statistic = MSR / MSE
    f_p_value = stats.f.sf(f_statistic, df_model, df_residuals)
    
    sigma_squared = MSE
    XtX_inv = np.linalg.inv(X.T @ X)
    se = np.sqrt(np.diag(sigma_squared * XtX_inv))
    t_values = beta / se
    p_values = 2 * (1 - stats.t.cdf(np.abs(t_values), df_residuals))
    
    skewness = stats.skew(residuals)
    kurtosis = stats.kurtosis(residuals, fisher=False)
    log_likelihood = -0.5 * n * (np.log(2 * np.pi * sigma_squared) + 1)
    AIC = 2 * k - 2 * log_likelihood
    BIC = n * np.log(SSE / n) + k * np.log(n)
    
    dw_statistic = np.sum(np.diff(residuals) ** 2) / SSE
    
    jarque_bera_stat = (n / 6) * (skewness**2 + (kurtosis - 3)**2 / 4)
    prob_jb = 1 - stats.chi2.cdf(jarque_bera_stat, df=2)
    
    return {
        'r_squared': r_squared,
        'Adjusted R-squared': adj_r_squared,
        'F-statistic': f_statistic,
        'Prob (F-statistic)': f_p_value,
        'Log-Likelihood': log_likelihood,
        'AIC': AIC,
        'BIC': BIC,
        'coef': beta,
        'std err': se,
        't': t_values,
        'P>|t|': p_values,
        'Skew': skewness,
        'Kurtosis': kurtosis,
        'Durbin-Watson': dw_statistic,
        'Jarque-Bera (JB)': jarque_bera_stat,
        'Prob(JB)': prob_jb
    }

### 1.4.1 Lineare Regression ohne Sparkoptimierung

In [27]:
def linear_regression_manual_qr(X, y):
    start_time = time.time()
    Q, R = manual_qr_decomposition(X)
    beta = backward_substitution(R, Q.T @ y)

    y_pred = np.dot(X, beta) # mit den durch QR berechneten Betas die vorhergesagten y-Werte des linReg Modells bestimmen
    residuals = y - y_pred

    result = compute_statistics(X, y, beta, residuals)
    
    end_time = time.time()
    elapsed_time = end_time - start_time

    return result, elapsed_time

### 1.4.2 Optimiert für Spark mit manueller QR Zerlegung

In [11]:
def fit_ols_manual(df):
    """
    Fit OLS using manual QR decomposition and backward substitution, with extended statistics.
    """
    start_time = time.time()
    
    # Step 1: Calculate combined QR decomposition across partitions
    def process_partition(iterator):
        # Local arrays for QR decomposition
        X_local = []
        y_local = []
        for row in iterator:
            X_local.append(row.features.toArray())
            y_local.append(row.y)
        
        # Early exit if partition is empty
        if not X_local:
            return []
        
        X_local = np.vstack(X_local)
        y_local = np.array(y_local)
        
        # Perform local QR decomposition
        Q_local, R_local = manual_qr_decomposition(X_local)
        
        # Calculate local Q^T y
        Qty_local = np.dot(Q_local.T, y_local)
        
        return [(R_local, Qty_local)]
    
    # Aggregate QR decomposition results
    results = df.rdd.mapPartitions(process_partition).reduce(lambda a, b: (
        a[0] + b[0],  # Sum R matrices
        a[1] + b[1]   # Sum Q^T y vectors
    ))
    
    R_total, Qty_total = results
    
    # Step 2: Solve for beta using the aggregated QR components
    beta = backward_substitution(R_total, Qty_total)
    
    # Step 3: Prepare full dataset X and y to compute statistics
    X = np.vstack([row.features.toArray() for row in df.collect()])
    y = np.array([row.y for row in df.collect()])
    y_pred = X.dot(beta)
    residuals = y - y_pred
    
    # Compute extended statistics using the unified function
    stats = compute_statistics(X, y, beta, residuals)
    
    computation_time = time.time() - start_time
    stats['computation_time'] = computation_time
    stats['n_samples'] = len(y)
    stats['n_features'] = X.shape[1]
    
    return beta, stats

### Optimiert für Spark mit Built-In Functions für die QR Zerlegung

In [12]:
def fit_ols_spark(df):
    """
    Fit an OLS model using Spark's built-in LinearRegression model, which uses QR decomposition.
    
    Args:
        df: Spark DataFrame with features and target variable.
    
    Returns:
        model: Fitted linear regression model
        training_summary: Summary of training metrics
    """
    lr = LinearRegression(featuresCol="features", labelCol="y", solver="normal")
    model = lr.fit(df)
    
    # Get the training summary to extract information like R^2, RMSE, etc.
    training_summary = model.summary
    
    return model, training_summary


## 1.5) Funktion für Durchführung des Benchmarks

### Berechnung für Numpy

In [13]:
def run_benchmark_numpy(n_list, beta_true, p, repetitions=10):
    results = []

    for n in n_list:
        times = []
        for _ in range(repetitions):
            X, y = create_data_numpy(n, p, beta_true)
            beta, elapsed_time = linear_regression_manual_qr(X, y)
            times.append(elapsed_time)

        avg_time = np.mean(times)
        std_time = np.std(times)
        results.append([n, avg_time, std_time])

    return results

### Berechnung für eine Range

In [14]:

def run_benchmark_range(max_n, p, beta_true, step_size=100000, repetitions=10):
    """Run benchmarking with fixed step size increments."""
    #n_values = np.arange(step_size, max_n + step_size, step_size, dtype=int)
    n_values = np.logspace(5, np.log10(max_n), 5, dtype=int)
    results = []
    
    for n in n_values:
        times = []
        r_squares = []
        successful_runs = 0
        
        print(f"\nBenchmarking n={n:,} samples...")
        
        for i in range(repetitions):
            try:
                df = create_data_spark(n, p, beta_true)
                start_time = time.time()
                beta, summary = fit_ols_manual(df)
                elapsed_time = time.time() - start_time
                
                times.append(elapsed_time)
                r_squares.append(summary['r_squared'])
                successful_runs += 1
                
                print(f"  Run {i+1}: time={elapsed_time:.2f}s, R²={summary['r_squared']:.4f}")
                
            except Exception as e:
                print(f"  Run {i+1} failed: {str(e)}")
        
        if successful_runs > 0:
            result = {
                'n_samples': n,
                'n_features': p,
                'mean_time': np.mean(times),
                'std_time': np.std(times),
                'mean_r_squared': np.mean(r_squares),
                'std_r_squared': np.std(r_squares),
                'successful_runs': successful_runs
            }
            results.append(result)
            6
            print(f"\nSummary for n={n:,}:")
            print(f"  Mean time: {result['mean_time']:.2f}s ± {result['std_time']:.2f}s")
            print(f"  Mean R²: {result['mean_r_squared']:.4f} ± {result['std_r_squared']:.4f}")
    
    return pd.DataFrame(results)

### Berechnung für einen festen Wert

In [15]:

def run_benchmark_point(n, p, beta_true, repetitions=10):
    """Run benchmarking for a single sample size n."""
    results = []
    times = []
    r_squares = []
    successful_runs = 0
    
    print(f"\nBenchmarking n={n:,} samples...")
    
    for i in range(repetitions):
        try:
            df = create_data_spark(n, p, beta_true)
            start_time = time.time()
            beta, summary = fit_ols_manual(df)
            elapsed_time = time.time() - start_time
            
            times.append(elapsed_time)
            r_squares.append(summary['r_squared'])
            successful_runs += 1
            
            print(f"  Run {i+1}: time={elapsed_time:.2f}s, R²={summary['r_squared']:.4f}")
            
        except Exception as e:
            print(f"  Run {i+1} failed: {str(e)}")
    
    if successful_runs > 0:
        result = {
            'n_samples': n,
            'n_features': p,
            'mean_time': np.mean(times),
            'std_time': np.std(times),
            'mean_r_squared': np.mean(r_squares),
            'std_r_squared': np.std(r_squares),
            'successful_runs': successful_runs
        }
        results.append(result)
        
        print(f"\nSummary for n={n:,}:")
        print(f"  Mean time: {result['mean_time']:.2f}s ± {result['std_time']:.2f}s")
        print(f"  Mean R²: {result['mean_r_squared']:.4f} ± {result['std_r_squared']:.4f}")
    
    return pd.DataFrame(results)


### Berechnung für Spark Functions

In [16]:
def run_benchmark_spark(max_n, p, beta_true, repetitions=10):
    """
    Run benchmarking for the OLS regression with different sample sizes.
    
    Args:
        max_n: Maximum number of samples
        p: Number of features
        beta_true: True coefficient values for the data generation
        repetitions: Number of repetitions for each sample size
    
    Returns:
        DataFrame: Benchmarking results as Pandas DataFrame
    """
    n_values = np.linspace(100000, max_n, 10, dtype=int)  # Minimum of 10 different sample sizes
    results = []

    for n in n_values:
        times = []
        for _ in range(repetitions):
            df = generate_synthetic_data(n, p, beta_true)
            start_time = time.time()
            model, summary = fit_ols_spark(df)
            elapsed_time = time.time() - start_time
            times.append(elapsed_time)

        avg_time = np.mean(times)
        std_time = np.std(times)
        results.append({
            'n_samples': n,
            'n_features': p,
            'mean_time': avg_time,
            'std_time': std_time,
            'r_squared': summary.r2,
            'adj_r_squared': summary.r2adj
        })

        print(f"Benchmark for n={n}: mean_time={avg_time:.4f}s, std_time={std_time:.4f}s, R^2={summary.r2:.4f}")

    return pd.DataFrame(results)

# 2) Durchführung des Benchmarks und Ausgeben der Ergebnisse

## 2.1) Durchführung

In [28]:
n_values = [200000, 500000, 1000000, 5000000, 10000000]
n_features = 7
beta_true = np.random.randn(n_features + 1)
results_numpy = run_benchmark_numpy(n_list=n_values, beta_true=beta_true, p=n_features) 

KeyboardInterrupt: 

### Benchmark Point

In [18]:
n_features = 7
beta_true = np.random.randn(n_features + 1)  # +1 for intercept
results_point = run_benchmark_point(n=1000, p=n_features, beta_true=beta_true)


Benchmarking n=1,000 samples...


  Run 1: time=5.74s, R²=-1.0447


  Run 2: time=1.96s, R²=-1.0613


  Run 3: time=1.74s, R²=-1.0284


  Run 4: time=1.76s, R²=-1.1200


  Run 5: time=1.67s, R²=-1.2264


  Run 6: time=1.54s, R²=-1.0896


  Run 7: time=1.71s, R²=-1.0860


  Run 8: time=1.62s, R²=-1.0211


  Run 9: time=1.54s, R²=-1.0442
  Run 10: time=1.65s, R²=-1.1579

Summary for n=1,000:
  Mean time: 2.09s ± 1.22s
  Mean R²: -1.0880 ± 0.0613


In [19]:
results_point

,n_samples,n_features,mean_time,std_time,mean_r_squared,std_r_squared,successful_runs
0,1000,7,2.093461,1.221278,-1.087952,0.061319,10


### Benchmark Range

In [20]:
n_features = 7
beta_true = np.random.randn(n_features + 1)  # +1 for intercept
results_range = run_benchmark_range(max_n=1000, p=n_features, beta_true=beta_true, step_size=100000)


Benchmarking n=100,000 samples...


  Run 1: time=7.32s, R²=-0.9896


  Run 2: time=5.28s, R²=-0.9933


  Run 3: time=5.41s, R²=-0.9965


  Run 4: time=5.10s, R²=-0.9933


...

  Run 5: time=1.56s, R²=-1.1033


  Run 6: time=1.62s, R²=-1.0539
  Run 7: time=1.61s, R²=-0.7653


  Run 8: time=1.68s, R²=-0.8743


  Run 9: time=1.61s, R²=-1.1935


  Run 10: time=1.61s, R²=-1.0543

Summary for n=1,000:
  Mean time: 1.58s ± 0.06s
  Mean R²: -1.0033 ± 0.1243


### Benchmark Spark

In [ ]:
max_n = 10000000
n_features = 7
beta_true = np.random.randn(n_features + 1)
results_spark = run_benchmark_spark(max_n, n_features, beta_true)


## 2.2) Ausgeben der Ergebnisse

In [22]:
import pandas as pd

def extract_info(dataframes):
    # Erstellen einer Liste, um die extrahierten Daten zu sammeln
    extracted_data = []
    
    # Durchlaufen aller DataFrames in der Liste
    for df in dataframes:
        # Überprüfen, ob die benötigten Spalten im DataFrame vorhanden sind
        if all(col in df.columns for col in ['n_samples', 'n_features', 'mean_time']):
            # Extrahieren der benötigten Daten und Hinzufügen zur Liste
            extracted_data.append(df[['n_samples', 'n_features', 'mean_time']])
        else:
            # Falls nicht alle Spalten vorhanden sind, fügen wir eine Fehlermeldung hinzu
            extracted_data.append(pd.DataFrame({'Error': ['Missing columns']}))
    
    # Kombinieren aller Daten in einem DataFrame
    result = pd.concat(extracted_data, ignore_index=True)
    return result



In [24]:
result_df = extract_info([results_numpy, results_range])
print(result_df)

   n_samples  n_features  mean_time
0       1000           7   2.093461
1     100000           7   5.533020
2      31622           7   2.720579
3      10000           7   1.897306
4       3162           7   1.697407
5       1000           7   1.575247


- bei n_samples

## 2.3) Plotten der Ergebnisse

In [ ]:
def plot_benchmark_results(n_values, avg_times, std_times):
    """
    Plots benchmark results as error bars without connecting lines.

    Parameters:
    n_values (list): List of sample sizes (number of observations).
    avg_times (list): List of average computation times for each sample size.
    std_times (list): List of standard deviations of the computation times for each sample size.
    """
    plt.figure(figsize=(10, 6))
    plt.errorbar(n_values, avg_times, yerr=std_times, fmt='o', ecolor='r', capsize=5,
                 label='Durchschnittliche Rechenzeit mit Standardabweichung', linestyle='None')
    plt.xscale('log')
    plt.yscale('log')
    plt.xlabel('Anzahl der Beobachtungen (n)')
    plt.ylabel('Rechenzeit (s)')
    plt.title('Rechenzeit der QR-Dekompositions-basierten Regression für wachsendes n')
    plt.grid(True)
    plt.legend()
    plt.show()

### Numpy

In [ ]:

plot_benchmark_results(n_values, avg_times, std_times)

### Spark manuell

### Spark normal

# 3) Vergleich mit statmodels

## 3.1) statmodels Ausgabe

In [ ]:
X, y = create_data(10000, 7, [-8, -1.6, 4.1, -10, -9.2, 1.3, 1.6, 2.3])
model = sm.OLS(y, X)
results = model.fit()

print(results.summary())

## 3.2) Ausgabe der implementierten Methode

In [ ]:
manual, _ = linear_regression_manual_qr(X,y)

print("\nErgebnisse der manuellen Berechnung:\n")

for key, value in manual[0].items():
    print(f"{key}: {value}")

print("-" * 50)  # 50 Zeichen lange Linie

for key, value in manual[1].items():
    print(f"{key}: {value}")

print("-" * 50)  # 50 Zeichen lange Linie

df = pd.DataFrame(manual[2])
print(df)

print("-" * 50)  # 50 Zeichen lange Linie

for key, value in manual[3].items():
    print(f"{key}: {value}")